# Calculate initial WSU data properties

The goal of this notebook is to calculate the properties of the initial WSU data based on the original data base with Early and Later WSU.

Record of changes:
* Original code (8/16/2024)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, QTable
import astropy.units as u
import pandas as pd
import pickle
from importlib import reload
import wsu_plots
import wsu_db
import re

## Read in data files

In [33]:
result_c7_mous = QTable.read('data/wsu_datarates_mit_per_mous_cycle7_20231212_without_band3_nbin3_npol4.ecsv')
result_c8_mous = QTable.read('data/wsu_datarates_mit_per_mous_cycle8_20231212_without_band3_nbin3_npol4.ecsv')

In [34]:
len(result_c7_mous)

2712

In [35]:
len(result_c8_mous)

2480

In [36]:
wsu_all = vstack([result_c7_mous,result_c8_mous])

In [37]:
len(wsu_all)

5192

In [38]:
wsu_all['weights_all'] = wsu_all['time_tot']/np.sum(wsu_all['time_tot'])

In [39]:
wsu_all.columns

<TableColumns names=('mous','proposal_id','schedblock_name','array','science_keyword','scientific_category','scientific_category_proposal','nant_typical','nant_array','nant_all','band','ntarget','s_fov','s_resolution','mosaic','imsize','pb','cell','blc_npol','blc_nspw','blc_specwidth','blc_freq','blc_velres','blc_nchan_agg','blc_nchan_max','blc_bandwidth_max','blc_bandwidth_agg','wsu_freq','wsu_npol','wsu_bandwidth_early','wsu_bandwidth_later_2x','wsu_bandwidth_later_4x','wsu_bandwidth_spw','wsu_nspw_early','wsu_nspw_later_2x','wsu_nspw_later_4x','wsu_specwidth_finest','wsu_chanavg_finest','wsu_velres_finest','wsu_specwidth_stepped','wsu_chanavg_stepped','wsu_velres_stepped','wsu_specwidth_stepped2','wsu_chanavg_stepped2','wsu_velres_stepped2','wsu_tint','wsu_nchan_spw_finest','wsu_nchan_spw_stepped','wsu_nchan_spw_stepped2','wsu_nchan_agg_finest_early','wsu_nchan_agg_finest_later_2x','wsu_nchan_agg_finest_later_4x','wsu_nchan_agg_stepped_early','wsu_nchan_agg_stepped_later_2x','wsu_nc

In [40]:
wsu_all['wsu_tint']

<Quantity [9.984, 9.984, 9.984, ..., 3.072, 3.072, 9.984] s>

## calculating the initial WSU properties

In [135]:
reload(wsu_db)
test = wsu_db.create_initial_wsu_db(wsu_all)

In [136]:
test.columns

<TableColumns names=('mous','proposal_id','schedblock_name','array','science_keyword','scientific_category','scientific_category_proposal','nant_typical','nant_array','nant_all','band','ntarget','s_fov','s_resolution','mosaic','imsize','pb','cell','blc_npol','blc_nspw','blc_specwidth','blc_freq','blc_velres','blc_nchan_agg','blc_nchan_max','blc_bandwidth_max','blc_bandwidth_agg','wsu_freq','wsu_npol','wsu_bandwidth_early','wsu_bandwidth_later_2x','wsu_bandwidth_later_4x','wsu_bandwidth_spw','wsu_nspw_early','wsu_nspw_later_2x','wsu_nspw_later_4x','wsu_specwidth_finest','wsu_chanavg_finest','wsu_velres_finest','wsu_specwidth_stepped','wsu_chanavg_stepped','wsu_velres_stepped','wsu_specwidth_stepped2','wsu_chanavg_stepped2','wsu_velres_stepped2','wsu_tint','wsu_nchan_spw_finest','wsu_nchan_spw_stepped','wsu_nchan_spw_stepped2','wsu_nchan_agg_finest_early','wsu_nchan_agg_finest_later_2x','wsu_nchan_agg_finest_later_4x','wsu_nchan_agg_stepped_early','wsu_nchan_agg_stepped_later_2x','wsu_nc

In [137]:
test['wsu_specwidth_stepped2']

<Quantity [  607.5,   661.5,   661.5, ..., 11529. , 11664. ,  7762.5] kHz>

hmmm.. it looks like in the earlier work, I'm not completely enforcing the band dependent channel averaging. Should I fix this? But this is the case where the velocity resolution is less than 0.1km/s, so allowed for in my earlier models.

In [138]:
idx = test['band'] == 5
print(np.min(test['wsu_specwidth_stepped2'][idx]))
print(np.max(test['wsu_specwidth_stepped2'][idx]))
print(np.min(test['wsu_specwidth_stepped2_initial'][idx]))
print(np.max(test['wsu_specwidth_stepped2_initial'][idx]))

27.0 kHz
6763.5 kHz
81.0 kHz
6763.5 kHz


In [139]:
idx = test['band'] == 5
print(np.min(test['wsu_chanavg_stepped2'][idx]))
print(np.max(test['wsu_chanavg_stepped2'][idx]))
print(np.min(test['wsu_chanavg_stepped2_initial'][idx]))
print(np.max(test['wsu_chanavg_stepped2_initial'][idx]))

2.0
501.0
6.0
501.0


In [140]:
idx = test['band'] == 5
print(np.min(test['wsu_velres_stepped2'][idx]))
print(np.max(test['wsu_velres_stepped2'][idx]))
print(np.min(test['wsu_velres_stepped2_initial'][idx]))
print(np.max(test['wsu_velres_stepped2_initial'][idx]))

0.04019781835382012 km / s
9.999640960363058 km / s
0.11831509142158976 km / s
9.999640960363058 km / s


In [141]:
idx = test['band'] == 5
print(np.min(test['wsu_nchan_spw_stepped2'][idx]))
print(np.max(test['wsu_nchan_spw_stepped2'][idx]))
print(np.min(test['wsu_nchan_spw_stepped2_initial'][idx]))
print(np.max(test['wsu_nchan_spw_stepped2_initial'][idx]))

295.0
74074.0
295.0
24691.0


In [142]:
idx = test['band'] == 5
print(np.min(test['wsu_nchan_agg_stepped2_early'][idx]))
print(np.max(test['wsu_nchan_agg_stepped2_early'][idx]))
print(np.min(test['wsu_nchan_agg_stepped2_initial'][idx]))
print(np.max(test['wsu_nchan_agg_stepped2_initial'][idx]))

1180.0
296296.0
1180.0
98764.0


In [143]:
print(np.min(test['wsu_nchan_agg_stepped2_initial']))
print(np.max(test['wsu_nchan_agg_stepped2_initial']))

484.0
148144.0
